In [69]:
import pandas as pd
import numpy as np
import biom
from biom import load_table
from biom.util import biom_open
import qiime2 as q2
import csv
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from scipy.stats import pearsonr
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [70]:
# Read in BIOM table
biom_path = '../tables/tables_rs210/per_genome/Genus_collapsed.biom'
biom_table = load_table(biom_path)
df = pd.DataFrame(biom_table.to_dataframe())
df = df.transpose() # transpose so features are columns
df.head()

,g__Abiotrophia,g__Acidovorax,g__Acinetobacter,g__Actinomadura,g__Actinomyces,g__Aerococcus,g__Aeromonas,g__Aggregatibacter,g__Alloiococcus,g__Alloprevotella,...,g__Thiobacillus,g__Triavirus,g__Tsukamurella,g__Urinicoccus,g__Varibaculum,g__Vedamuthuvirus,g__Veillonella,g__Vimunumvirus,g__Xanthomonas,g__Yonseivirus
15443.209.S78.L005,0.0,0.0,89.0,0.0,112.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,216.0,0.0,0.0,0.0
15443.136.S29.L005,29.0,8.0,11.0,0.0,398.0,0.0,1.0,2.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,2.0,0.0
15443.167.S39.L005,0.0,0.0,3.0,0.0,947.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
15443.160.S33.L005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15443.192.S61.L005,3.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0


In [71]:
# Check any samples with no count data and filter them out
rows_with_sum_zero = df.sum(axis=1) == 0
df_zero_sum = df[rows_with_sum_zero]
print(f"Rows where the sum of values is 0: {df_zero_sum.index}")

# Drop that sample from the dataframe 
df = df.drop(df_zero_sum.index)

Rows where the sum of values is 0: Index(['15443.178.S48.L005', '15443.179.S49.L005'], dtype='object')


In [72]:
# Edit sample names, convert index to string (if not already)
df.index = df.index.astype(str)
# Transform the index by keeping only the part between the first and third periods
df.index = [x.split('.')[1] for x in df.index]
df.head()

,g__Abiotrophia,g__Acidovorax,g__Acinetobacter,g__Actinomadura,g__Actinomyces,g__Aerococcus,g__Aeromonas,g__Aggregatibacter,g__Alloiococcus,g__Alloprevotella,...,g__Thiobacillus,g__Triavirus,g__Tsukamurella,g__Urinicoccus,g__Varibaculum,g__Vedamuthuvirus,g__Veillonella,g__Vimunumvirus,g__Xanthomonas,g__Yonseivirus
209,0.0,0.0,89.0,0.0,112.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,216.0,0.0,0.0,0.0
136,29.0,8.0,11.0,0.0,398.0,0.0,1.0,2.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,2.0,0.0
167,0.0,0.0,3.0,0.0,947.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
192,3.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0


In [73]:
# Read in metadata
metadata_path = '../metadata/metadata.csv'
md = pd.read_csv(metadata_path)
md['sample_ID'] = md['sample_ID'].astype(str)
md.head()

,patient_number,sample_ID,subject_ID,microbiome_type,body_site,AD_status,les_or_nonles,age_months,age_years,sex,skin_group
0,1,72,1,human_skin,forehead,healthy,NL,5,0,M,healthy
1,1,73,1,human_skin,nose,healthy,NL,5,0,M,healthy
2,1,75,1,human_skin,knee_pit,healthy,NL,5,0,M,healthy
3,2,76,2,human_skin,elbow_pit,healthy,NL,2,0,F,healthy
4,2,78,2,human_skin,forehead,healthy,NL,2,0,F,healthy


In [74]:
# Kepp only human_skin samples (remove human_oral samples)
skin_md = md[md['microbiome_type'] == 'human_skin']
skin_sample_ids = skin_md['sample_ID'].values
filtered_df = df.loc[df.index.isin(skin_sample_ids)]
filtered_df.head()

,g__Abiotrophia,g__Acidovorax,g__Acinetobacter,g__Actinomadura,g__Actinomyces,g__Aerococcus,g__Aeromonas,g__Aggregatibacter,g__Alloiococcus,g__Alloprevotella,...,g__Thiobacillus,g__Triavirus,g__Tsukamurella,g__Urinicoccus,g__Varibaculum,g__Vedamuthuvirus,g__Veillonella,g__Vimunumvirus,g__Xanthomonas,g__Yonseivirus
209,0.0,0.0,89.0,0.0,112.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,216.0,0.0,0.0,0.0
136,29.0,8.0,11.0,0.0,398.0,0.0,1.0,2.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,2.0,0.0
167,0.0,0.0,3.0,0.0,947.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
192,3.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0


In [75]:
# Collapsing to top 20 genera and adding an 'Other' category for the rest
top_genera = filtered_df.sum().sort_values(ascending=False).head(20).index.tolist()
df_top_genera = filtered_df[top_genera]
df_top_genera[' s__Other'] = filtered_df.drop(columns=top_genera).sum(axis=1)
df_top_genera = df_top_genera.transpose()
df_top_genera.head()

/var/folders/gg/tjf8z7kd7351mxr7fbw1s6j00000gp/T/ipykernel_73882/2493314149.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_genera[' s__Other'] = filtered_df.drop(columns=top_genera).sum(axis=1)


,209,136,167,160,192,189,78,162,163,193,...,151,159,218,214,215,185,212,79,81,75
g__Cutibacterium,2246.0,356.0,157.0,30.0,1139.0,1529.0,1518.0,2057.0,603.0,919.0,...,630.0,1371.0,985.0,968.0,1005.0,32.0,933.0,1645.0,22.0,70.0
g__Streptococcus,1250.0,1116.0,1112.0,6.0,1282.0,397.0,937.0,1036.0,2861.0,1178.0,...,876.0,351.0,764.0,166.0,1032.0,2617.0,298.0,1046.0,3323.0,2920.0
g__Malassezia,379.0,37.0,86.0,1.0,2070.0,407.0,1098.0,108.0,24.0,1572.0,...,34.0,47.0,460.0,124.0,169.0,3.0,429.0,221.0,1.0,6.0
g__Staphylococcus,26.0,37.0,11.0,4794.0,97.0,247.0,164.0,420.0,34.0,168.0,...,9.0,19.0,169.0,84.0,55.0,29.0,37.0,247.0,280.0,215.0
g__Corynebacterium,47.0,293.0,61.0,3.0,80.0,241.0,402.0,89.0,28.0,164.0,...,113.0,166.0,277.0,306.0,226.0,59.0,113.0,468.0,1.0,1.0


In [76]:
# Sort samples based on Cutibacterium relative abundances
cutibacterium_values = df_top_genera.loc[' g__Cutibacterium']
sorted_columns = cutibacterium_values.sort_values(ascending=False).index
df_top_genera_Cuti_sorted = df_top_genera[sorted_columns]
df_top_genera_Cuti_sorted.head()

,112,114,123,110,124,202,73,203,120,201,...,157,172,185,160,204,93,177,87,81,76
g__Cutibacterium,4837.0,4253.0,4095.0,4055.0,3855.0,3841.0,3824.0,3686.0,3164.0,2897.0,...,41.0,36.0,32.0,30.0,26.0,25.0,23.0,22.0,22.0,20.0
g__Streptococcus,6.0,16.0,139.0,10.0,174.0,162.0,135.0,241.0,327.0,209.0,...,5.0,4.0,2617.0,6.0,599.0,4171.0,2933.0,3394.0,3323.0,2.0
g__Malassezia,87.0,489.0,266.0,390.0,195.0,37.0,772.0,19.0,911.0,44.0,...,8.0,0.0,3.0,1.0,0.0,0.0,4.0,0.0,1.0,0.0
g__Staphylococcus,4.0,8.0,10.0,39.0,8.0,9.0,29.0,19.0,4.0,42.0,...,272.0,4816.0,29.0,4794.0,0.0,85.0,1028.0,277.0,280.0,19.0
g__Corynebacterium,8.0,97.0,43.0,73.0,58.0,37.0,95.0,33.0,98.0,90.0,...,4210.0,1.0,59.0,3.0,0.0,0.0,51.0,0.0,1.0,4735.0


In [77]:
# Convert to relative abundance table
total_counts_per_sample = df_top_genera_Cuti_sorted.sum(axis=0)
relative_abundance_df = df_top_genera_Cuti_sorted.div(total_counts_per_sample, axis=1)
# Drop 'Other' 
relative_abundance_df.head()

,112,114,123,110,124,202,73,203,120,201,...,157,172,185,160,204,93,177,87,81,76
g__Cutibacterium,0.973631,0.858325,0.828780,0.833162,0.790283,0.783078,0.773619,0.748730,0.648228,0.606194,...,0.008349,0.007316,0.006535,0.006090,0.005241,0.005696,0.004752,0.005186,0.005195,0.004036
g__Streptococcus,0.001208,0.003229,0.028132,0.002055,0.035670,0.033028,0.027311,0.048954,0.066994,0.043733,...,0.001018,0.000813,0.534409,0.001218,0.120742,0.950330,0.605992,0.800094,0.784652,0.000404
g__Malassezia,0.017512,0.098688,0.053835,0.080131,0.039975,0.007543,0.156180,0.003859,0.186642,0.009207,...,0.001629,0.000000,0.000613,0.000203,0.000000,0.000000,0.000826,0.000000,0.000236,0.000000
g__Staphylococcus,0.000805,0.001615,0.002024,0.008013,0.001640,0.001835,0.005867,0.003859,0.000820,0.008788,...,0.055386,0.978663,0.005922,0.973203,0.000000,0.019367,0.212397,0.065299,0.066116,0.003835
g__Corynebacterium,0.001610,0.019576,0.008703,0.014999,0.011890,0.007543,0.019219,0.006703,0.020078,0.018832,...,0.857259,0.000203,0.012048,0.000609,0.000000,0.000000,0.010537,0.000000,0.000236,0.955600


In [78]:
# Transpose so it is samples by features
relative_abundance_df = relative_abundance_df.transpose()

In [79]:
# Get the order of samples in metadata file so that skin_group info can be correctly tied
sample_order = list(set(md['sample_ID']))

In [80]:
# Reorder the dataframe rows to be in same order as metadata
filtered_order = [index for index in sample_order if index in relative_abundance_df.index]
df_reordered = relative_abundance_df.loc[filtered_order]

df_reordered

,g__Cutibacterium,g__Streptococcus,g__Malassezia,g__Staphylococcus,g__Corynebacterium,g__Rothia,g__Neisseria,g__Actinomyces,g__Haemophilus,g__Veillonella,...,g__Cytomegalovirus,g__Pseudomonas,g__Prevotella,g__Mycolicibacterium,g__Bifidobacterium,g__Porphyromonas,g__Meiothermus,g__Bradyrhizobium,g__Klebsiella,s__Other
185,0.006535,0.534409,0.000613,0.005922,0.012048,0.030427,0.069226,0.004697,0.043496,0.072289,...,0.000000,0.001634,0.104350,0.000000,0.000204,0.025117,0.000204,0.000000,0.000000,0.086379
90,0.202575,0.018306,0.757795,0.004426,0.001811,0.000201,0.000805,0.000805,0.000604,0.000402,...,0.000000,0.000402,0.000201,0.000201,0.000000,0.000000,0.000201,0.006840,0.000000,0.004225
209,0.458461,0.255154,0.077363,0.005307,0.009594,0.011023,0.000000,0.022862,0.000204,0.044091,...,0.000000,0.005920,0.058583,0.001225,0.000000,0.000000,0.000204,0.000000,0.000612,0.031231
96,0.322756,0.346973,0.054280,0.003549,0.013779,0.015031,0.055741,0.010647,0.016910,0.016701,...,0.000000,0.001044,0.013779,0.001044,0.000418,0.006263,0.000000,0.000000,0.000000,0.117328
204,0.005241,0.120742,0.000000,0.000000,0.000000,0.689982,0.001008,0.003628,0.099375,0.000000,...,0.000000,0.000000,0.000202,0.000202,0.000000,0.064100,0.000202,0.000000,0.000000,0.015118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,0.514437,0.037210,0.252338,0.004880,0.109191,0.011387,0.000000,0.044937,0.000407,0.000203,...,0.002440,0.001017,0.002237,0.000203,0.000000,0.000610,0.008337,0.000000,0.000000,0.009963
79,0.348665,0.221704,0.046842,0.052353,0.099195,0.016532,0.002331,0.014837,0.003603,0.009538,...,0.000000,0.008690,0.003603,0.002120,0.002755,0.001060,0.005723,0.051293,0.004027,0.101738
120,0.648228,0.066994,0.186642,0.000820,0.020078,0.011063,0.003073,0.007171,0.000615,0.008605,...,0.000000,0.000615,0.015571,0.001024,0.000410,0.002868,0.001434,0.000000,0.000205,0.023970
117,0.056347,0.194386,0.012359,0.001885,0.103268,0.077922,0.127147,0.132803,0.014244,0.013825,...,0.000000,0.001885,0.007122,0.024927,0.006284,0.042941,0.002095,0.000209,0.000628,0.177210


In [81]:
# Make into numpy array for plotting and pearson calculations
g__Cutibacterium = df_reordered[' g__Cutibacterium'].to_numpy()
g__Streptococcus = df_reordered[' g__Streptococcus'].to_numpy()

In [82]:
# Define color map
color_map = {'healthy': 'blue', 'AD_pos_L': 'red', 'AD_pos_NL': 'orange'}

plt.figure(figsize=(6, 6))

# Since we're only plotting 'healthy', we can simplify the legend
plt.legend(handles=[mpatches.Patch(color=color_map['healthy'], label='healthy')], bbox_to_anchor=(0.95, 0.9))  # Example starting point

# plt.title('Correlation between Streptoccocus and Cutibacterium in AD non-lesional samples', size = 10)
plt.ylabel('g__Streptococcus')
plt.xlabel('g__Cutibacterium')

# Calculate and plot the trend line only for healthy data
# First, filter data for healthy
ad_pos_cutibacterium = [cutibacterium for skin_group, cutibacterium in zip(md['skin_group'], g__Cutibacterium) if skin_group == 'healthy']
ad_pos_streptococcus = [streptococcus for skin_group, streptococcus in zip(md['skin_group'], g__Streptococcus) if skin_group == 'healthy']

# Calculate the trend line
z = np.polyfit(ad_pos_cutibacterium, ad_pos_streptococcus, 1)
p = np.poly1d(z)
# Sort the cutibacterium values for a smooth trend line
sorted_cutibacterium = np.sort(ad_pos_cutibacterium)
plt.plot(sorted_cutibacterium, p(sorted_cutibacterium), "r-")

# Calculate the Pearson correlation coefficient and p-value
correlation_coefficient, p_value = pearsonr(ad_pos_cutibacterium, ad_pos_streptococcus)

# Annotate the Pearson correlation coefficient and p-value
plt.text(0.92, 0.75, f'Pearson r: {correlation_coefficient:.2f}\np-value: {p_value:.2e}', transform=plt.gca().transAxes, ha='right', va='top', bbox=dict(boxstyle="round", alpha=0.5, color="w"))

# Use seaborn's regplot to plot the data with a regression line and confidence interval
sns.regplot(x=ad_pos_cutibacterium, y=ad_pos_streptococcus, color='blue', ci=95)

ax = plt.gca()
# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)

plt.savefig('../plots/correlation_plots/Cuti_vs_Strep_corr_healthy.png')


In [83]:
# Define color map
color_map = {'healthy': 'blue', 'AD_pos_L': 'red', 'AD_pos_NL': 'orange'}

plt.figure(figsize=(6, 6))

# Since we're only plotting 'healthy', we can simplify the legend
plt.legend(handles=[mpatches.Patch(color=color_map['AD_pos_L'], label='AD_pos_L')], bbox_to_anchor=(0.95, 0.9))  # Example starting point

# plt.title('Correlation between Streptoccocus and Cutibacterium in AD non-lesional samples', size = 10)
plt.ylabel('g__Streptococcus')
plt.xlabel('g__Cutibacterium')

# Calculate and plot the trend line only for healthy data
# First, filter data for healthy
ad_pos_cutibacterium = [cutibacterium for skin_group, cutibacterium in zip(md['skin_group'], g__Cutibacterium) if skin_group == 'AD_pos_L']
ad_pos_streptococcus = [streptococcus for skin_group, streptococcus in zip(md['skin_group'], g__Streptococcus) if skin_group == 'AD_pos_L']

# Calculate the trend line
z = np.polyfit(ad_pos_cutibacterium, ad_pos_streptococcus, 1)
p = np.poly1d(z)
# Sort the cutibacterium values for a smooth trend line
sorted_cutibacterium = np.sort(ad_pos_cutibacterium)
plt.plot(sorted_cutibacterium, p(sorted_cutibacterium), "r-")

# Calculate the Pearson correlation coefficient and p-value
correlation_coefficient, p_value = pearsonr(ad_pos_cutibacterium, ad_pos_streptococcus)

# Annotate the Pearson correlation coefficient and p-value
plt.text(0.92, 0.75, f'Pearson r: {correlation_coefficient:.2f}\np-value: {p_value:.2e}', transform=plt.gca().transAxes, ha='right', va='top', bbox=dict(boxstyle="round", alpha=0.5, color="w"))

# Use seaborn's regplot to plot the data with a regression line and confidence interval
sns.regplot(x=ad_pos_cutibacterium, y=ad_pos_streptococcus, color='red', ci=95)

ax = plt.gca()
# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)

plt.savefig('../plots/correlation_plots/Cuti_vs_Strep_corr_AD_pos_L.png')


In [84]:
# Assuming the necessary data and variables (md, g__Cutibacterium, g__Streptococcus) are defined

# Define color map
color_map = {'healthy': 'blue', 'AD_pos_L': 'red', 'AD_pos_NL': 'orange'}

plt.figure(figsize=(6, 6))

# Since we're only plotting 'healthy', we can simplify the legend
plt.legend(handles=[mpatches.Patch(color=color_map['AD_pos_NL'], label='AD_pos_NL')], bbox_to_anchor=(0.95, 0.9))  # Example starting point

# plt.title('Correlation between Streptoccocus and Cutibacterium in AD non-lesional samples', size = 10)
plt.ylabel('g__Streptococcus')
plt.xlabel('g__Cutibacterium')

# Calculate and plot the trend line only for healthy data
# First, filter data for healthy
ad_pos_cutibacterium = [cutibacterium for skin_group, cutibacterium in zip(md['skin_group'], g__Cutibacterium) if skin_group == 'AD_pos_NL']
ad_pos_streptococcus = [streptococcus for skin_group, streptococcus in zip(md['skin_group'], g__Streptococcus) if skin_group == 'AD_pos_NL']

# Calculate the trend line
z = np.polyfit(ad_pos_cutibacterium, ad_pos_streptococcus, 1)
p = np.poly1d(z)
# Sort the cutibacterium values for a smooth trend line
sorted_cutibacterium = np.sort(ad_pos_cutibacterium)
plt.plot(sorted_cutibacterium, p(sorted_cutibacterium), "r-")

# Calculate the Pearson correlation coefficient and p-value
correlation_coefficient, p_value = pearsonr(ad_pos_cutibacterium, ad_pos_streptococcus)

# Annotate the Pearson correlation coefficient and p-value
plt.text(0.92, 0.75, f'Pearson r: {correlation_coefficient:.2f}\np-value: {p_value:.2e}', transform=plt.gca().transAxes, ha='right', va='top', bbox=dict(boxstyle="round", alpha=0.5, color="w"))

# Use seaborn's regplot to plot the data with a regression line and confidence interval
sns.regplot(x=ad_pos_cutibacterium, y=ad_pos_streptococcus, color='orange', ci=95)

ax = plt.gca()
# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)

plt.savefig('../plots/correlation_plots/Cuti_vs_Strep_corr_AD_pos_NL.png')
